In [19]:
!pip3 install spacy >= 2.2.4

ERROR: Could not find a version that satisfies the requirement 2.2.4
ERROR: No matching distribution found for 2.2.4


In [20]:
import string

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import spacy 
import praw
import streamlit as st


%matplotlib inline

In [21]:

reddit = praw.Reddit(client_id='qQBQxY9R1zGrYJH9pACBOw', client_secret='MjrjCout_Omkh_3feZFq3UrzyLBicw', user_agent='Crypto')


In [22]:
crypto = []
ml_subreddit = reddit.subreddit('Cryptocurrency')
for post in ml_subreddit.hot(limit=30000):
    crypto.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
crypto = pd.DataFrame(crypto,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])

In [23]:
crypto.dropna(subset=['body'], inplace=True)

In [24]:
crypto['original_body'] = crypto['body']

In [34]:
crypto

,title,score,id,subreddit,url,num_comments,body,created,original_body
0,"Daily Discussion - November 19, 2021 (GMT+0)",202,qx37qc,CryptoCurrency,https://www.reddit.com/r/CryptoCurrency/commen...,10274,welcome daily discussion read disclaimer guide...,1.637280e+09,**Welcome to the Daily Discussion. Please read...
1,I absolutely do not sympathize with people who...,1037,qxepgd,CryptoCurrency,https://www.reddit.com/r/CryptoCurrency/commen...,466,friend lost 2 k squid game token fiasco later ...,1.637323e+09,My friend lost $2K in the whole squid game tok...
2,Should Loopring be in your portfolio?,455,qxflkv,CryptoCurrency,https://www.reddit.com/r/CryptoCurrency/commen...,386,holding loopring awhile sadly bought currently...,1.637326e+09,I have been holding Loopring for awhile now an...
3,Housing Prices Increased 18% in 6 Months: Cryp...,240,qxg4zr,CryptoCurrency,https://www.reddit.com/r/CryptoCurrency/commen...,155,area housing prices went 18 6 months picture n...,1.637328e+09,In my area housing prices went up 18% in just ...
4,Hillary Clinton will mark the start of this Bu...,223,qxgirf,CryptoCurrency,https://www.reddit.com/r/CryptoCurrency/commen...,257,hilary opinion huge favour disclousure main re...,1.637330e+09,"So Hilary in my opinion, just made us a huge f..."
...,...,...,...,...,...,...,...,...,...
679,700 million dollars says crypto will be around...,12,qwoqov,CryptoCurrency,https://www.reddit.com/r/CryptoCurrency/commen...,19,thinking crazy number sure naming rights agree...,1.637239e+09,I’ve been thinking about how crazy that number...
680,New Record For Bitcoin Lightning Network As Ad...,6,qwtswr,CryptoCurrency,https://www.newsbtc.com/news/bitcoin/bitcoin-l...,7,,1.637254e+09,
681,A DAO Has Raised $45.6M to Buy the U.S. Consti...,6,qwtqdw,CryptoCurrency,https://cryptobriefing.com/a-dao-has-raised-45...,6,,1.637254e+09,
682,Comment your coin . Lets share some positivity,7,qwv20m,CryptoCurrency,https://www.reddit.com/r/CryptoCurrency/commen...,38,guy think experienced dip scary feeling seeing...,1.637257e+09,"So, guy i think i just experienced my first di..."


In [25]:
import spacy
nlp = spacy.blank('en')


import re

def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

def remove_stop_words(text):
    doc = nlp(text)
    nlp.Defaults.stop_words -= {"nt","crypto", "cryptocurrency"}
    return " ".join([token.text for token in doc if not token.is_stop])

def lemmatize_words(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])

remove_spaces = lambda x : re.sub('\\s+', ' ', x)

# Reference : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

remove_double_quotes = lambda x : x.replace('"', '')
remove_single_quotes = lambda x : x.replace('\'', '')
trim = lambda x : x.strip()

other_chars = ['*', '#', '&x200B', '[', ']', '; ',' ;' "&nbsp", "“","“","”", "x200b"]


def remove_other_chars(x: str):
    for char in other_chars:
        x = x.replace(char, '')
    
    return x


def lower_case_text(text):
    return text.lower()

funcs = [
    remove_urls, 
    remove_punctuation,
    remove_stop_words, 
    remove_emoji, 
    remove_double_quotes, 
    remove_single_quotes,
    lower_case_text,
    remove_other_chars,
    # lemmatize_words,
    remove_spaces,
    trim]



In [26]:
for fun in funcs:
    crypto['body'] = crypto['body'].apply(fun)

In [27]:
crypto.reset_index(inplace=True)
crypto.drop(['index'], axis=1, inplace=True)



In [28]:
body_list = crypto.body.tolist()

In [29]:
def generate_ngrams(text, n_gram=2):
    token = [token for token in text.lower().split(' ') if token != '']
    ngrams = zip(*[token[i:] for i in range(n_gram)])
    return [' '.join(ngram) for ngram in ngrams]

In [30]:
!pip3 install fbpca

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import linalg
from sklearn import decomposition
import fbpca

number_of_topics = 30
num_top_words = 20
vectorizer = TfidfVectorizer()

In [37]:
bigram_vectorizer = TfidfVectorizer(ngram_range=(2,2))

In [38]:
bigrams_vectors = bigram_vectorizer.fit_transform(body_list).todense()
bigrams_vocab = np.array(bigram_vectorizer.get_feature_names())

# LDA

In [39]:
docs = [body.split() for body in body_list]

In [40]:
from gensim.corpora import Dictionary

dic = Dictionary(docs)

2021-11-19 18:59:14.924 INFO    gensim.corpora.dictionary: adding document #0 to Dictionary(0 unique tokens: [])
2021-11-19 18:59:14.981 INFO    gensim.corpora.dictionary: built Dictionary(8291 unique tokens: ['2fa', 'acting', 'actions', 'afford', 'age']...) from 684 documents (total 48596 corpus positions)
2021-11-19 18:59:15.007 INFO    gensim.utils: Dictionary lifecycle event {'msg': "built Dictionary(8291 unique tokens: ['2fa', 'acting', 'actions', 'afford', 'age']...) from 684 documents (total 48596 corpus positions)", 'datetime': '2021-11-19T18:59:14.982242', 'gensim': '4.1.2', 'python': '3.8.8 (default, Apr 13 2021, 12:59:45) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'created'}


In [41]:
corpus = [dic.doc2bow(doc) for doc in docs]

## Training 

Now it's time to train our topic model. We do this with the following parameters:

- **corpus**: the bag-of-word representations of our documents
- **id2token**: the mapping from indices to words
- **num_topics**: the number of topics we want the model to identify
- **chunksize**: the number of documents the model sees for every update
- **passes**: the number of times we show the total corpus to the model during training
- **random_state**: we use a seed to ensure reproducibility.

In [42]:
from gensim.models import LdaModel

model = LdaModel(corpus=corpus, id2word=dic, num_topics=number_of_topics, chunksize=2500, passes=5, random_state=1)

2021-11-19 18:59:23.509 INFO    gensim.models.ldamodel: using symmetric alpha at 0.03333333333333333
2021-11-19 18:59:23.511 INFO    gensim.models.ldamodel: using symmetric eta at 0.03333333333333333
2021-11-19 18:59:23.516 INFO    gensim.models.ldamodel: using serial LDA version on this node
2021-11-19 18:59:23.545 INFO    gensim.models.ldamodel: running online (multi-pass) LDA training, 30 topics, 5 passes over the supplied corpus of 684 documents, updating model once every 684 documents, evaluating perplexity every 684 documents, iterating 50x with a convergence threshold of 0.001000
2021-11-19 18:59:23.546 WARNING gensim.models.ldamodel: too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2021-11-19 18:59:23.870 INFO    gensim.models.ldamodel: -19.919 per-word bound, 991029.1 perplexity estimate based on a held-out corpus of 684 documents with 48596 words
2021-11-19 18:59:23.870 INFO    gensim.models.ldamodel: PRO

In [46]:
model.print_topics()

2021-11-19 19:04:27.885 INFO    gensim.models.ldamodel: topic #10 (0.033): 0.025*"nt" + 0.010*"think" + 0.009*"m" + 0.008*"people" + 0.006*"crypto" + 0.005*"going" + 0.005*"like" + 0.005*"believe" + 0.005*"new" + 0.005*"price"
2021-11-19 19:04:27.887 INFO    gensim.models.ldamodel: topic #23 (0.033): 0.028*"value" + 0.022*"portfolio" + 0.019*"total" + 0.016*"€" + 0.015*"market" + 0.015*"investment" + 0.014*"profits" + 0.013*"monetary" + 0.013*"month" + 0.012*"taking"
2021-11-19 19:04:27.888 INFO    gensim.models.ldamodel: topic #5 (0.033): 0.013*"cro" + 0.012*"value" + 0.011*"ethereum" + 0.010*"defi" + 0.009*"blockchain" + 0.008*"use" + 0.007*"card" + 0.007*"token" + 0.006*"algorand" + 0.006*"app"
2021-11-19 19:04:27.890 INFO    gensim.models.ldamodel: topic #17 (0.033): 0.008*"crypto" + 0.008*"wallet" + 0.006*"tether" + 0.006*"report" + 0.006*"nt" + 0.006*"buy" + 0.006*"like" + 0.005*"…" + 0.004*"going" + 0.004*"market"
2021-11-19 19:04:27.891 INFO    gensim.models.ldamodel: topic #20

[(10,
  '0.025*"nt" + 0.010*"think" + 0.009*"m" + 0.008*"people" + 0.006*"crypto" + 0.005*"going" + 0.005*"like" + 0.005*"believe" + 0.005*"new" + 0.005*"price"'),
 (23,
  '0.028*"value" + 0.022*"portfolio" + 0.019*"total" + 0.016*"€" + 0.015*"market" + 0.015*"investment" + 0.014*"profits" + 0.013*"monetary" + 0.013*"month" + 0.012*"taking"'),
 (5,
  '0.013*"cro" + 0.012*"value" + 0.011*"ethereum" + 0.010*"defi" + 0.009*"blockchain" + 0.008*"use" + 0.007*"card" + 0.007*"token" + 0.006*"algorand" + 0.006*"app"'),
 (17,
  '0.008*"crypto" + 0.008*"wallet" + 0.006*"tether" + 0.006*"report" + 0.006*"nt" + 0.006*"buy" + 0.006*"like" + 0.005*"…" + 0.004*"going" + 0.004*"market"'),
 (20,
  '0.012*"crypto" + 0.008*"earn" + 0.008*"card" + 0.007*"people" + 0.007*"like" + 0.007*"good" + 0.006*"m" + 0.006*"nfts" + 0.006*"project" + 0.005*"worth"'),
 (22,
  '0.013*"countries" + 0.010*"m" + 0.009*"cbdc" + 0.008*"crypto" + 0.007*"digital" + 0.006*"prediction" + 0.006*"end" + 0.006*"98k" + 0.006*"incom

[]

In [43]:
for (topic, words) in model.print_topics():
    print(topic+1, ":", words, '\n\n')

2021-11-19 18:59:34.515 INFO    gensim.models.ldamodel: topic #26 (0.033): 0.016*"auction" + 0.011*"buy" + 0.011*"nt" + 0.010*"2021" + 0.010*"dip" + 0.009*"crypto" + 0.009*"2022" + 0.007*"time" + 0.007*"dec" + 0.007*"good"
2021-11-19 18:59:34.516 INFO    gensim.models.ldamodel: topic #18 (0.033): 0.012*"nt" + 0.011*"eth" + 0.010*"like" + 0.009*"nano" + 0.009*"people" + 0.008*"ve" + 0.007*"fees" + 0.007*"crypto" + 0.006*"gas" + 0.006*"bank"
2021-11-19 18:59:34.517 INFO    gensim.models.ldamodel: topic #5 (0.033): 0.013*"cro" + 0.012*"value" + 0.011*"ethereum" + 0.010*"defi" + 0.009*"blockchain" + 0.008*"use" + 0.007*"card" + 0.007*"token" + 0.006*"algorand" + 0.006*"app"
2021-11-19 18:59:34.519 INFO    gensim.models.ldamodel: topic #12 (0.033): 0.033*"2021" + 0.023*"crypto" + 0.017*"experiment" + 0.014*"january" + 0.011*"sp" + 0.010*"month" + 0.010*"index" + 0.010*"portfolio" + 0.009*"500" + 0.009*"october"
2021-11-19 18:59:34.520 INFO    gensim.models.ldamodel: topic #17 (0.033): 0.008

27 : 0.016*"auction" + 0.011*"buy" + 0.011*"nt" + 0.010*"2021" + 0.010*"dip" + 0.009*"crypto" + 0.009*"2022" + 0.007*"time" + 0.007*"dec" + 0.007*"good" 


19 : 0.012*"nt" + 0.011*"eth" + 0.010*"like" + 0.009*"nano" + 0.009*"people" + 0.008*"ve" + 0.007*"fees" + 0.007*"crypto" + 0.006*"gas" + 0.006*"bank" 


6 : 0.013*"cro" + 0.012*"value" + 0.011*"ethereum" + 0.010*"defi" + 0.009*"blockchain" + 0.008*"use" + 0.007*"card" + 0.007*"token" + 0.006*"algorand" + 0.006*"app" 


13 : 0.033*"2021" + 0.023*"crypto" + 0.017*"experiment" + 0.014*"january" + 0.011*"sp" + 0.010*"month" + 0.010*"index" + 0.010*"portfolio" + 0.009*"500" + 0.009*"october" 


18 : 0.008*"crypto" + 0.008*"wallet" + 0.006*"tether" + 0.006*"report" + 0.006*"nt" + 0.006*"buy" + 0.006*"like" + 0.005*"…" + 0.004*"going" + 0.004*"market" 


9 : 0.010*"binance" + 0.009*"information" + 0.009*"bitcoin" + 0.008*"market" + 0.007*"share" + 0.006*"use" + 0.006*"transaction" + 0.006*"fees" + 0.006*"way" + 0.005*"read" 


15 : 0.015*

In [ ]:
original_body_list = crypto.original_body.tolist()

In [ ]:
for (text, doc) in zip(original_body_list[:9], docs[:9]):
    print('\033[1m' + 'Text: ' + '\033[0m', text)
    print('\033[1m' + 'Topics: ' + '\033[0m', [(topic+1, prob) for (topic, prob) in model[dic.doc2bow(doc)] if prob > 0.15])
    print('\n')

Text:  **Welcome to the Daily Discussion. Please read the disclaimer, guidelines, and rules before participating.**
 
***
-
    
###Disclaimer:

Consider all information posted here with several liberal heaps of salt, and always cross check any information you may read on this thread with known sources. Any trade information posted in this open thread may be highly misleading, and could be an attempt to manipulate new readers by known "pump and dump (PnD) groups" for their own profit. BEWARE of such practices and exercise utmost caution before acting on any trade tip mentioned here. 


**Please be careful about what information you share and the actions you take.** Do not share the amounts of your portfolios (why not just share percentage?). Do not share your private keys or wallet seed. Use strong, non-SMS 2FA if possible. Beware of scammers and be smart. Do not invest more than you can afford to lose, and do not fall for pyramid schemes, promises of unrealistic returns (get-rich-quic

Now let's see what topic is going to assign to the post-**2179**, previously tested with the svd decomposition for the unigram term-document matrix.

In [45]:
model[dic.doc2bow(docs[])]

[(20, 0.9859879)]

In [ ]:
print('\033[1m' + 'Text: ' + '\033[0m', original_body_list[20])
print('\033[1m' + 'Topic: ' + '\033[0m', [(topic+1, prob) for (topic, prob) in model[dic.doc2bow(docs[20])] if prob > 0.1])

Text:  About a month ago Turkeys president has declared a "War on Crypto" following young turkish people using Crypto to escape the extreme inflation.

You may ask how this war on crypto is going?

Turkeys currency has just hit an inflation of 20%, with no end in sight.


Following his statement, even more turkish people are now using Crypto to escape the inflation, seems like the war is going in reverse for him.

[(source on inflation)](https://www.tagesschau.de/wirtschaft/weltwirtschaft/erdogan-lira-tuerkei-101.html)
Topic:  [(2, 0.9758294)]


the topics assigned by the LDA are not 100% spot-on, **topic 5** seems to be appropriate but **topic 9** seems to be out of place.

Let's see what kind of topic the LDA is going to choose for the post **280** tested with the bigram matrix decomposition.

In [ ]:
print('\033[1m' + 'Text: ' + '\033[0m', original_body_list[30])
print('\033[1m' + 'Topic: ' + '\033[0m',[(topic+1, prob) for (topic, prob) in model[dic.doc2bow(docs[30])] if prob > 0.1])

Text:  My DCA day (pay day) is still half a month away and greed literally burns me from the inside because I think I can miss this dip.

I always try to catch the moment and buy at the lowest possible price, and it's a damn shame to watch such a great opportunity to load my bags (before price skyrocket) pass me by.

How do you guys deal with the feeling of lost profits at such moments? I understand that I have no choice but to just relax and continue watching the charts, but it's kinda difficult

p.s. looks like my love for crypto is becoming an addiction lol
Topic:  [(4, 0.9817565)]


In [ ]:
from textblob import TextBlob
# create a function to get subjectivity
def getSubjectivity(twt):
    return TextBlob(twt).sentiment.subjectivity

# create a function to get the polarity
def getPolarity(twt):
    return TextBlob(twt).sentiment.polarity

# create two new columns called "Subjectivity" & "Polarity"
crypto['subjectivity'] = crypto['original_body'].apply(getSubjectivity)
crypto['polarity'] = crypto['original_body'].apply(getPolarity)

In [ ]:
# create a function get the sentiment text
def getSentiment(score):
    if score < 0:
        return "negative"
    elif score == 0:
        return "neutral"
    else:
        return "positive"

In [ ]:
# create a column to store the text sentiment
crypto['sentiment'] = crypto['polarity'].apply(getSentiment)



,title,score,id,subreddit,url,num_comments,body,created,original_body,subjectivity,polarity,sentiment
0,"Daily Discussion - November 19, 2021 (GMT+0)",186,qx37qc,CryptoCurrency,https://www.reddit.com/r/CryptoCurrency/commen...,8124,welcome daily discussion read disclaimer guide...,1.637280e+09,**Welcome to the Daily Discussion. Please read...,0.430163,0.094759,positive
1,What's the next metaverse play with high ROI l...,871,qx73x6,CryptoCurrency,https://www.reddit.com/r/CryptoCurrency/commen...,567,okay ton research … 3k spend tomorrow kind wan...,1.637292e+09,Okay just did a ton of research… I have $3k to...,0.476463,0.129577,positive
2,ADOPTION: The richest person in crypto chooses...,281,qxbosh,CryptoCurrency,https://www.reddit.com/r/CryptoCurrency/commen...,171,sam bankmanfried ceo alameda research chosen c...,1.637310e+09,Sam Bankman-Fried who is the CEO of Alameda Re...,0.525571,0.178671,positive
3,I absolutely do not sympathize with people who...,140,qxepgd,CryptoCurrency,https://www.reddit.com/r/CryptoCurrency/commen...,133,friend lost 2 k squid game token fiasco later ...,1.637323e+09,My friend lost $2K in the whole squid game tok...,0.480895,0.090294,positive
4,Now is your chance to actually buy the dip,3174,qwxint,CryptoCurrency,https://www.reddit.com/r/CryptoCurrency/commen...,1148,says late beg dip chance asked dollar cost ave...,1.637264e+09,Everyone says they’re “late” and beg for a dip...,0.413333,0.153333,positive
...,...,...,...,...,...,...,...,...,...,...,...,...
655,Is current state of crypto currency a prelude ...,7,qwp793,CryptoCurrency,https://www.reddit.com/r/CryptoCurrency/commen...,17,crypto saga began saw effects unbridled greed ...,1.637240e+09,The crypto saga began after we saw the effects...,0.414867,0.123556,positive
656,IOTEX 2.0: The Rise of MachineFi,12,qwmxnz,CryptoCurrency,https://www.reddit.com/r/CryptoCurrency/commen...,13,iotex actually night announced machinefi follo...,1.637231e+09,"IOTEX have just (actually, last night) announc...",0.610309,0.130712,positive
657,This is why $AVAX is killing the Market right ...,4,qwrbry,CryptoCurrency,/r/Avax/comments/qw5ce1/this_is_why_avax_is_ki...,1,,1.637247e+09,,0.000000,0.000000,neutral
658,How can I/we help you? Vol. 2,2,qwxvgp,CryptoCurrency,https://www.reddit.com/r/CryptoCurrency/commen...,21,previous post similar nature got attention lot...,1.637265e+09,My [previous](https://old.reddit.com/r/CryptoC...,0.480208,-0.005208,negative


In [ ]:
if __name__ == ‘__main__‘:
    main()
    import warnings
    warnings.filterwarnings(‘ignore’)